# Comparing ecoinvent and EXIOBASE
Part 1) Matching dict preparation

Part 2) Matching

## Scenario name

In [1]:
scenario = '2019-12-02_WW'  # must be the same as the HIOT Excel sheetname

# Part 1) Matching dict preparation

In [2]:
import pandas as pd
import pickle
import numpy as np
import os

## Files

In [3]:
DATA_PATH = "../data/"
SCENARIO_PATH = "../results/" + scenario + "/"
print(SCENARIO_PATH)

if not os.path.exists(SCENARIO_PATH):
    os.makedirs(SCENARIO_PATH)
    os.makedirs(SCENARIO_PATH + 'exiobase/')
    os.makedirs(SCENARIO_PATH + 'matching/')
    os.makedirs(SCENARIO_PATH + 'matching_results/')
    os.makedirs(SCENARIO_PATH + 'figures/')
    os.makedirs(SCENARIO_PATH + 'tables/')

../results/2019-12-02_WW/


In [4]:
# ecoinvent
activity_overview = f"{DATA_PATH}ecoinvent/activity_overview_3.4_cut-off_LCIA_results.xlsx"

# EXIOBASE
hiot_results = f"{DATA_PATH}exiobase/hiot_results_PTM_v2.xlsx"
#hiot_results_sheet = "v_3_3_18_2019-11-22"
hiot_results_sheet = scenario

# Matching
product_matching = f"{DATA_PATH}matching/match_v4.xlsx"
product_matching_sheet = 'concordance'
process_matching = f"{DATA_PATH}matching/match_v2_electricity.xlsx"
region_matching = f"{DATA_PATH}matching/matching_ecoinvent_Exiobase_geographies.xlsx"
region_matching_sheet = "matched_191117"

# Load data

### ecoinvent activity overview

In [5]:
AO = pd.read_excel(activity_overview)
#ao.set_index(0, inplace=True)
print(AO.shape)
AO.head(2)

(14889, 37)


,key,#,id,filename,activityName,geography,startDate,endDate,specialActivityType,technologyLevel,...,unitName,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"('IPCC 2013', 'climate change', 'GWP 100a')_all_CFs","('IPCC 2013', 'climate change', 'GWP 100a')"
0,"('ecoinvent 3.4 cutoff', '0008dcb8309bd0ca4820...",3712,400461dd-45f4-4bcc-8eae-afa68e57664a,400461dd-45f4-4bcc-8eae-afa68e57664a_2966d161-...,market group for heavy fuel oil,RER,2015-01-01,2017-12-31,market group,Current,...,kg,1.019820e+11,True,NaN,0.0000,allocatable product,non-mft,1,0.47094,0.470319
1,"('ecoinvent 3.4 cutoff', '001145735c7cee63ae10...",402,0766970d-6c54-497e-b458-2626f56d9e0c,0766970d-6c54-497e-b458-2626f56d9e0c_45fbbc41-...,"treatment of municipal solid waste, incineration",AT,2006-01-01,2017-12-31,ordinary transforming activity,Current,...,MJ,4.823674e+09,True,NaN,0.0106,allocatable product,non-mft,1,0.00000,0.000000


### exiobase regionalized sectors

In [6]:
#EX = pd.read_excel(f"{PATH}exiobase/results.xlsx")
EX = pd.read_excel(hiot_results, sheet_name=hiot_results_sheet)
EX = EX[EX.columns.drop(list(EX.filter(regex='Unnamed')))]
print(EX.shape)
EX.head(2)

(8200, 10)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit
0,1,AU,Cultivation of paddy rice,c01.a,P_PARI,tonnes,1.595365,kg,1.595365,kg CO2-eq/kg
1,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.306648,kg,0.306648,kg CO2-eq/kg


#### drop first row, which is data description

In [7]:
EX.drop([0], axis=0, inplace=True)
print(EX.shape)
EX.head(2)

(8199, 10)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit
1,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.306648,kg,0.306648,kg CO2-eq/kg
2,3,AU,Cultivation of cereal grains nec,c01.c,P_OCER,tonnes,0.266411,kg,0.266411,kg CO2-eq/kg


# Helper Functions

## Dictionaries to help matching

In [8]:
def map_data_from_column(DF, key_column="ISIC", value_column=None):
    """Map one or more values from the value_column to one unique value in the key_column."""
    unique_elements = DF[key_column].unique()
    print("Mapping", value_column if value_column else "Indices", "values to", len(unique_elements), "unique", key_column, "keys.")
    if not value_column:  # then return the indices of the DF
        return {ue: DF[DF[key_column] == ue].index.tolist() for ue in unique_elements}
    else:
        return {ue: DF[DF[key_column] == ue][value_column].tolist() for ue in unique_elements}

# Product matching

In [9]:
pm = pd.read_excel(product_matching, sheet_name=product_matching_sheet, skiprows=[0, 2])
pm.drop(labels=["Unnamed: 0", "Unnamed: 2",  102, "Comments", "Unnamed: 204"], axis=1, inplace=True)
pm.set_index("Unnamed: 1", inplace=True)
pm.index.names = ['product']
pm.fillna(0, inplace=True)
print(pm.shape)
pm.head(2)

(2851, 200)


,Paddy rice,Wheat,Cereal grains nec,"Vegetables, fruit, nuts",Oil seeds,"Sugar cane, sugar beet",Plant-based fibers,Crops nec,Cattle,Pigs,...,Paper for treatment: landfill,Plastic waste for treatment: landfill,Inert/metal/hazardous waste for treatment: landfill,Textiles waste for treatment: landfill,Wood waste for treatment: landfill,Membership organisation services n.e.c.,"Recreational, cultural and sporting services",Other services,Private households with employed persons,Extra-territorial organizations and bodies
product,,,,,,,,,,,,,,,,,,,,,
"heat and power co-generation unit, 160kW electrical, common components for heat+electricity",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"heat and power co-generation unit, 1MW electrical, common components for heat+electricity",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
product_EI_to_EX = {}
for index, row in pm.iterrows():
    #print(index)
    corresponding_sectors = [x for x in row.index[row.to_numpy().nonzero()]]
    #print(corresponding_sectors)
    product_EI_to_EX.update({row.name: corresponding_sectors})
#product_EI_to_EX

In [11]:
product_EX_to_EI = {}
for column in pm.columns:
    #print(column)
    corresponding_sectors = [x for x in pm.index[pm[column].to_numpy().nonzero()]]
    #print(corresponding_sectors)
    product_EX_to_EI.update({column: corresponding_sectors})
#product_EX_to_EI

In [12]:
with open(f'{SCENARIO_PATH}matching/product_EX_to_EI.pickle', 'wb') as handle:
    pickle.dump(product_EX_to_EI, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
with open(f'{SCENARIO_PATH}matching/product_EI_to_EX.pickle', 'wb') as handle:
    pickle.dump(product_EI_to_EX, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Process matching (for electricity sector)

In [14]:
ppm = pd.read_excel(process_matching, skiprows=[0, 2])
ppm.drop(labels=["Unnamed: 0", 
                 "Unnamed: 2",  
                 "Unnamed: 3",  
                 "Unnamed: 4",  
                 "Unnamed: 5",  
                 "Unnamed: 6",  
                 "Unnamed: 7",  
                 "Unnamed: 8",
                 "Comments",
                 12,
                ], axis=1, inplace=True)
ppm.set_index("Unnamed: 1", inplace=True)
ppm.index.names = ['key']
ppm.fillna(0, inplace=True)
print(ppm.shape)
ppm.head(2)

(14889, 201)


,Paddy rice,Wheat,Cereal grains nec,"Vegetables, fruit, nuts",Oil seeds,"Sugar cane, sugar beet",Plant-based fibers,Crops nec,Cattle,Pigs,...,Plastic waste for treatment: landfill,Inert/metal/hazardous waste for treatment: landfill,Textiles waste for treatment: landfill,Wood waste for treatment: landfill,Membership organisation services n.e.c.,"Recreational, cultural and sporting services",Other services,Private households with employed persons,Extra-territorial organizations and bodies,Unnamed: 210
key,,,,,,,,,,,,,,,,,,,,,
"('ecoinvent 3.4 cutoff', '60274947cf82e6633d8c8488b7029c34')",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"('ecoinvent 3.4 cutoff', '7dd298333a92144698cfa6ee16325dbc')",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
process_EI_to_EX = {}
for index, row in ppm.iterrows():
    #print(index)
    corresponding_sectors = [x for x in row.index[row.to_numpy().nonzero()]]
    #print(corresponding_sectors)
    process_EI_to_EX.update({row.name: corresponding_sectors})#
#process_EI_to_EX

In [16]:
process_EX_to_EI = {}
for column in ppm.columns:
    #print(column)
    corresponding_sectors = [x for x in ppm.index[ppm[column].to_numpy().nonzero()]]
    #print(corresponding_sectors)
    process_EX_to_EI.update({column: corresponding_sectors})
#process_EX_to_EI

In [17]:
with open(f'{SCENARIO_PATH}matching/process_EX_to_EI.pickle', 'wb') as handle:
    pickle.dump(process_EX_to_EI, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
with open(f'{SCENARIO_PATH}matching/process_EI_to_EX.pickle', 'wb') as handle:
    pickle.dump(process_EI_to_EX, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Region matching

In [19]:
GEO = pd.read_excel(region_matching, sheet_name=region_matching_sheet, skiprows=[0])
print(GEO.shape)
GEO.head(2)

(261, 11)


,#,# Activities,ecoinvent 3.4,geography name,geography shortcut,ecoinvent long name,Continent,Exiobase 3,Alternative,Matching type,Comment
0,1,5,AE,Asia,United Arab Emirates,United Arab Emirates,Asia,WM,NaN,EXIOBASE broader,NaN
1,2,8,AL,Albania,Albania,Albania,Europe,WE,NaN,EXIOBASE broader,NaN


In [20]:
geo_EX_to_EI = map_data_from_column(GEO, key_column="ecoinvent 3.4", value_column="Exiobase 3")
#geo_EX_to_EI

Mapping Exiobase 3 values to 261 unique ecoinvent 3.4 keys.


In [21]:
geo_EI_to_EX = map_data_from_column(GEO, key_column="Exiobase 3", value_column="ecoinvent 3.4")
#geo_EI_to_EX

Mapping ecoinvent 3.4 values to 52 unique Exiobase 3 keys.


In [22]:
with open(f'{SCENARIO_PATH}matching/geo_EX_to_EI.pickle', 'wb') as handle:
    pickle.dump(geo_EX_to_EI, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
with open(f'{SCENARIO_PATH}matching/geo_EI_to_EX.pickle', 'wb') as handle:
    pickle.dump(geo_EI_to_EX, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Units

In [24]:
EX["PhysicalUnit"].unique()

array(['kg', 'MJ', 'Meuro'], dtype=object)

In [25]:
AO["unitName"].unique()

array(['kg', 'MJ', 'hour', 'm3', 'kWh', 'unit', 'metric ton*km', 'm',
       'm2', 'm2*year', 'ha', 'km', 'm*year', 'l', 'person*km', 'kg*day',
       'km*year'], dtype=object)

In [26]:
units_EI_to_EX = {
    "kg": ["kg"],  # l
    "MJ": ["MJ", "kWh"],
    "Meuro": [],
}

In [27]:
units_EX_to_EI = {
    "kg": ["kg"],  # l
    "MJ": ["MJ"],
    "kWh": ["MJ"],
}

In [28]:
with open(f'{SCENARIO_PATH}matching/units_EI_to_EX.pickle', 'wb') as handle:
    pickle.dump(units_EI_to_EX, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
with open(f'{SCENARIO_PATH}matching/units_EX_to_EI.pickle', 'wb') as handle:
    pickle.dump(units_EX_to_EI, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Part 2) Matching

# Matching Exiobase - Ecoinvent

In [30]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:.3g}'.format
import numpy as np
import pickle
from itertools import islice
import random

import helper_functions as hf

import matplotlib.pyplot as plt
%matplotlib inline  

# Load data

In [31]:
def load_pickle(filename):
    with open(filename, 'rb') as handle:
        data = pickle.load(handle)
        print(len(data))
        if isinstance(data, dict):
            print(list(islice(data.items(), 2)))
    return data

In [32]:
# for updating the matching dictionaries to the miot equivalent of the hiot names
def replace_dict_keys(original_dict, key_map):
    new_dict = {}
    for key, value in original_dict.items():
        new_dict.update({
            key_map[key]: value
        })
    return new_dict

def replace_dict_values(original_dict, key_map):
    new_dict = {}
    for key, values in original_dict.items():
        new_dict.update({
            #key: [key_map.get(v, None) for v in values]
            key: [key_map[v] for v in values]
        })
    return new_dict

#### Product and process matching (from Bridge matrices)

In [33]:
product_EX_to_EI = load_pickle(f'{SCENARIO_PATH}matching/product_EX_to_EI.pickle')

200
[('Paddy rice', ['rice', 'rice seed, for sowing']), ('Wheat', ['wheat grain', 'wheat grain, feed', 'wheat grain, feed, organic', 'wheat grain, feed, Swiss integrated production', 'wheat grain, organic', 'wheat grain, Swiss integrated production', 'wheat seed, for sowing', 'wheat seed, organic, for sowing', 'wheat seed, Swiss integrated production, for sowing'])]


In [34]:
product_EI_to_EX = load_pickle(f'{SCENARIO_PATH}matching/product_EI_to_EX.pickle')

2851
[('heat and power co-generation unit, 160kW electrical, common components for heat+electricity', ['Construction work']), ('heat and power co-generation unit, 1MW electrical, common components for heat+electricity', ['Construction work'])]


In [35]:
process_EX_to_EI = load_pickle(f'{SCENARIO_PATH}matching/process_EX_to_EI.pickle')

201
[('Paddy rice', []), ('Wheat', [])]


In [36]:
process_EI_to_EX = load_pickle(f'{SCENARIO_PATH}matching/process_EI_to_EX.pickle')

14889
[("('ecoinvent 3.4 cutoff', '60274947cf82e6633d8c8488b7029c34')", []), ("('ecoinvent 3.4 cutoff', '7dd298333a92144698cfa6ee16325dbc')", [])]


#### Geographical matching

In [37]:
geo_EI_to_EX = load_pickle(f'{SCENARIO_PATH}matching/geo_EI_to_EX.pickle')

52
[('WM', ['AE', 'BH', 'IAI Area, Gulf Cooperation Council', 'IL', 'IQ', 'IR', 'JO', 'KW', 'LB', 'OM', 'QA', 'RME', 'SA', 'SY', 'YE']), ('WE', ['AL', 'AM', 'BA', 'GI', 'IAI Area, EU27 & EFTA', 'IS', 'ME', 'MK', 'RS', 'UA', 'WEU', 'XK'])]


In [38]:
print("Excluding RoW!")
geo_EI_to_EX["ROW"].pop()

Excluding RoW!


'RoW'

In [39]:
geo_EX_to_EI = load_pickle(f'{SCENARIO_PATH}matching/geo_EX_to_EI.pickle')

261
[('AE', ['WM']), ('AL', ['WE'])]


#### Unit matching

In [40]:
units_EX_to_EI = load_pickle(f'{SCENARIO_PATH}matching/units_EX_to_EI.pickle')

3
[('kg', ['kg']), ('MJ', ['MJ'])]


In [41]:
units_EI_to_EX = load_pickle(f'{SCENARIO_PATH}matching/units_EI_to_EX.pickle')

3
[('kg', ['kg']), ('MJ', ['MJ', 'kWh'])]


# Load EX-EI Dataframes for matching

#### Load ecoinvent Dataframe

#### Add ISIC level 1-5 codes and descriptions to AO

In [42]:
ISIC = pd.read_excel(f"{DATA_PATH}matching/ISIC_nb_activities_units_geographies_in_ecoinvent.xlsx")
#ao.set_index(0, inplace=True)
ISIC = ISIC.astype(str)
ISIC['#activities'] = ISIC['#activities'].astype(int)
#ISIC.rename(columns={"Code (ecoinvent)": "ISIC code"}, inplace=True)
ISIC.rename(columns={"ISIC": "ISIC v4"}, inplace=True)
print(ISIC.shape)
ISIC.head(2)

(186, 21)


,Unnamed: 0,ISIC v4,#activities,Code (ecoinvent),Description (ecoinvent),ISIC level,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
0,0,"0111:Growing of cereals (except rice), legumin...",205,0111,"Growing of cereals (except rice), leguminous c...",4,A,1.0,11.0,111.0,nan,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",Growing of non-perennial crops,"Growing of cereals (except rice), leguminous c...",nan,"A:Agriculture, forestry and fishing","01:Crop and animal production, hunting and rel...",011:Growing of non-perennial crops,"0111:Growing of cereals (except rice), legumin...",nan
1,1,0112:Growing of rice,8,0112,Growing of rice,4,A,1.0,11.0,112.0,nan,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",Growing of non-perennial crops,Growing of rice,nan,"A:Agriculture, forestry and fishing","01:Crop and animal production, hunting and rel...",011:Growing of non-perennial crops,0112:Growing of rice,nan


In [43]:
AO = pd.merge(AO, ISIC, on=["ISIC v4"], how="outer")
#AO = AO[AO['specialActivityType'] == 'ordinary transforming activity']  # removes one strange row only... 
print(AO.shape)
AO.head(2)

(14889, 57)


,key,#,id,filename,activityName,geography,startDate,endDate,specialActivityType,technologyLevel,inheritance status,parentActivityId,tags,synonyms,ISIC v4,ISIC code,ISIC name,specialty production,constrained market,accessRestrictedTo,dataEntryBy,dataGenerator,isActiveAuthor,group,name,CPC,amount,unitName,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"('IPCC 2013', 'climate change', 'GWP 100a')_all_CFs","('IPCC 2013', 'climate change', 'GWP 100a')",Unnamed: 0,#activities,Code (ecoinvent),Description (ecoinvent),ISIC level,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
0,"('ecoinvent 3.4 cutoff', '0008dcb8309bd0ca4820...",3712,400461dd-45f4-4bcc-8eae-afa68e57664a,400461dd-45f4-4bcc-8eae-afa68e57664a_2966d161-...,market group for heavy fuel oil,RER,2015-01-01,2017-12-31,market group,Current,allocated dataset,0d176784-ae74-458d-b75b-3369f0cac894,NaN,NaN,1920:Manufacture of refined petroleum products,1920,Manufacture of refined petroleum products,False,NaN,1,[System],[System],True,ReferenceProduct,heavy fuel oil,33370: Fuel oils n.e.c.,1,kg,1.02e+11,True,NaN,0,allocatable product,non-mft,1,0.471,0.47,64,158,1920,Manufacture of refined petroleum products,4,C,19.0,192.0,1920.0,nan,Manufacturing,Manufacture of coke and refined petroleum prod...,Manufacture of refined petroleum products,Manufacture of refined petroleum products,nan,C:Manufacturing,19:Manufacture of coke and refined petroleum p...,192:Manufacture of refined petroleum products,1920:Manufacture of refined petroleum products,nan
1,"('ecoinvent 3.4 cutoff', '048ac7e1860972d562e9...",7691,831198c8-1ddd-49ff-be80-9dce40d83a7e,831198c8-1ddd-49ff-be80-9dce40d83a7e_4cf8fef9-...,molecular sieve separation of naphtha,RER,1998-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,5e3ac209-6b8b-432d-8ce0-b6b52c110ad4,ConvertedDataset\n ConvertedDataset\n Converte...,NaN,1920:Manufacture of refined petroleum products,1920,Manufacture of refined petroleum products,False,NaN,1,Jürgen Sutter,Jürgen Sutter,False,ReferenceProduct,fraction 7 from naphtha separation,341: Basic organic chemicals,1,kg,9.94e+07,True,Calculated from production volume of reference...,0.522,allocatable product,non-mft,1,0.94,0.939,64,158,1920,Manufacture of refined petroleum products,4,C,19.0,192.0,1920.0,nan,Manufacturing,Manufacture of coke and refined petroleum prod...,Manufacture of refined petroleum products,Manufacture of refined petroleum products,nan,C:Manufacturing,19:Manufacture of coke and refined petroleum p...,192:Manufacture of refined petroleum products,1920:Manufacture of refined petroleum products,nan


In [44]:
# save to Excel
AO.to_excel(f"{DATA_PATH}ecoinvent/activity_overview_3.4_cut-off_LCIA_results_with_ISIC_details.xlsx")

#### Excluding markets

In [45]:
AO = AO[AO['specialActivityType'] == 'ordinary transforming activity']
print(AO.shape)

(10805, 57)


#### Excluding RoW

In [46]:
AO = AO[AO['geography'] != 'RoW']
print(AO.shape)

(7986, 57)


#### Load Exiobase DF

In [47]:
#EX = pd.read_excel(f"{PATH}exiobase/hiot_results.xlsx")
#EX = pd.read_excel(f"{PATH}exiobase/hiot_results_PTM_v2.xlsx", sheet_name="v_3_3_18_2019-11-22")
#EX.drop([0], axis=0, inplace=True)
EX['CarbonFootprint'].replace(0, np.nan, inplace=True)  # replace 0 values with nan (otherwise mean, etc. are disturbed)
print(EX.shape)
EX.dropna(inplace=True)
print(EX.shape)
EX.head(2)

(8199, 10)
(7012, 10)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit
1,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg
2,3,AU,Cultivation of cereal grains nec,c01.c,P_OCER,tonnes,0.266,kg,0.266,kg CO2-eq/kg


In [48]:
# #EX = pd.read_excel(f"{PATH}exiobase/hiot_results.xlsx")
# EX = pd.read_excel(f"{PATH}exiobase/hiot_results_PTM_v2_units_converted.xlsx", sheet_name="v_3_3_18")
# #EX.drop([0], axis=0, inplace=True)
# #EX['CarbonFootprint'].replace(0, np.nan, inplace=True)  # replace 0 values with nan (otherwise mean, etc. are disturbed)
# print(EX.shape)
# EX.dropna(inplace=True)
# print(EX.shape)
# EX.head(2)

### ProductTypeName is different in hiot; for consistency reasons with the miot, we convert the names back to the ones used in miot

In [49]:
pm = pd.read_excel(f"{DATA_PATH}matching/product_match.xlsx", sheet_name="product_aggregation")
print(pm.shape)
pm.head(2)

(200, 11)


,old_name,old_code,old_synonym,new_name,new_code,new_synonym,old_idx,new_idx,old_or_new,ISIC 1,our_name
0,Paddy rice,p01.a,C_PARI,Cultivation of paddy rice,c01.a,C_PARI,1,1,old,A,Paddy rice
1,Wheat,p01.b,C_WHEA,Cultivation of wheat,c01.b,C_WHEA,2,2,old,A,Wheat


In [50]:
# construct a dictionary from new(hiot) to old(miot) names
hiot_to_miot_dict = {}
miot_to_hiot_dict = {}
miot_to_new_miot = {}
for index, row in pm.iterrows():
    hiot_to_miot_dict.update({
        row["new_name"]: row["new_name"] if row["old_or_new"] == "new" else row["old_name"]
    })
    miot_to_hiot_dict.update({
        row["old_name"]: row["new_name"]
    })
    miot_to_new_miot.update(
    {
        row["old_name"]: row["new_name"] if row["old_or_new"] == "new" else row["old_name"]
    })
#hiot_to_miot_dict
print("Hiot to miot:", len(set(hiot_to_miot_dict.keys())), len(set(hiot_to_miot_dict.values())))
print("Miot to hiot:", len(set(miot_to_hiot_dict.keys())), len(set(miot_to_hiot_dict.values())))
print("Miot to new miot:", len(set(miot_to_new_miot.keys())), len(set(miot_to_new_miot.values())))

Hiot to miot: 164 164
Miot to hiot: 200 164
Miot to new miot: 200 164


#### Save MIOT to HIOT matching dicts

In [51]:
with open(f'{DATA_PATH}matching/hiot_to_miot_dict.pickle', 'wb') as handle:
    pickle.dump(hiot_to_miot_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'{DATA_PATH}matching/miot_to_hiot_dict.pickle', 'wb') as handle:
    pickle.dump(miot_to_hiot_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'{DATA_PATH}matching/miot_to_new_miot.pickle', 'wb') as handle:
    pickle.dump(miot_to_new_miot, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [52]:
# apply this map to EX
hiot_names = EX["ProductTypeName_of_hiot"]
EX["ProductTypeName"] = hiot_names.map(hiot_to_miot_dict)
print(EX.shape)
EX.head(2)

(7012, 11)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName
1,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg,Wheat
2,3,AU,Cultivation of cereal grains nec,c01.c,P_OCER,tonnes,0.266,kg,0.266,kg CO2-eq/kg,Cereal grains nec


In [53]:
# save to Excel
EX.to_excel(f"{SCENARIO_PATH}exiobase/hiot_results.xlsx")

#### Update also the matching dictionaries with the new miot names

In [54]:
product_EI_to_EX = replace_dict_values(product_EI_to_EX, miot_to_new_miot)
print(len(product_EI_to_EX), list(islice(product_EI_to_EX.items(), 2)))
process_EI_to_EX = replace_dict_values(process_EI_to_EX, miot_to_new_miot)
print(len(process_EI_to_EX), list(islice(process_EI_to_EX.items(), 2)))

2851 [('heat and power co-generation unit, 160kW electrical, common components for heat+electricity', ['Construction work']), ('heat and power co-generation unit, 1MW electrical, common components for heat+electricity', ['Construction work'])]
14889 [("('ecoinvent 3.4 cutoff', '60274947cf82e6633d8c8488b7029c34')", []), ("('ecoinvent 3.4 cutoff', '7dd298333a92144698cfa6ee16325dbc')", [])]


# Match EX and EI dataframes (Pandas way)

1. Create lists of tuples from matched sectors, geographies and units.
1. Then make DFs from each list
1. Then merge each of theses DFs with EX
1. the resulting DF can be merged with EI based on "product OR process & geo & unit"

In [55]:
def tuples_from_dict(matching_dict):
    tuples = []
    for key, values in matching_dict.items():
        for value in values:
            tuples.append((key, value))
    return tuples

In [56]:
tuples_from_dict(geo_EI_to_EX)[:3]

[('WM', 'AE'), ('WM', 'BH'), ('WM', 'IAI Area, Gulf Cooperation Council')]

In [57]:
# Products
DF_product_matching = pd.DataFrame(tuples_from_dict(product_EI_to_EX), columns=["name", "ProductTypeName"])
print(DF_product_matching.shape)
DF_product_matching.head(2)

(2327, 2)


,name,ProductTypeName
0,"heat and power co-generation unit, 160kW elect...",Construction work
1,"heat and power co-generation unit, 1MW electri...",Construction work


In [58]:
# Processes
DF_process_matching = pd.DataFrame(tuples_from_dict(process_EI_to_EX), columns=["key", "ProductTypeName"])
print(DF_process_matching.shape)
DF_process_matching.head(2)

(1987, 2)


,key,ProductTypeName
0,"('ecoinvent 3.4 cutoff', '22772a024d350dc0101b...",Electricity by biomass and waste
1,"('ecoinvent 3.4 cutoff', '888c4aa6b7c44a3fec5e...",Electricity by biomass and waste


In [59]:
# Geographies
DF_geo_matching = pd.DataFrame(tuples_from_dict(geo_EI_to_EX), columns=["CountryCode", "geography"])
print(DF_geo_matching.shape)
DF_geo_matching.head(2)

(260, 2)


,CountryCode,geography
0,WM,AE
1,WM,BH


In [60]:
# Units
DF_unit_matching = pd.DataFrame(tuples_from_dict(units_EI_to_EX), columns=["PhysicalUnit", "unitName"])
print(DF_unit_matching.shape)
DF_unit_matching.head()

(3, 2)


,PhysicalUnit,unitName
0,kg,kg
1,MJ,MJ
2,MJ,kWh


# Merging...

In [61]:
EX_geo = pd.merge(EX, DF_geo_matching, on=["CountryCode"], how="inner")
print(EX_geo.shape)
EX_geo.head(2)

(38042, 12)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography
0,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg,Wheat,AU
1,3,AU,Cultivation of cereal grains nec,c01.c,P_OCER,tonnes,0.266,kg,0.266,kg CO2-eq/kg,Cereal grains nec,AU


In [62]:
EX_geo_unit = pd.merge(EX_geo, DF_unit_matching, on=["PhysicalUnit"], how="inner")
print(EX_geo_unit.shape)
EX_geo_unit.head(2)

(31435, 13)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography,unitName
0,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg,Wheat,AU,kg
1,3,AU,Cultivation of cereal grains nec,c01.c,P_OCER,tonnes,0.266,kg,0.266,kg CO2-eq/kg,Cereal grains nec,AU,kg


In [63]:
EX_geo_unit_product = pd.merge(EX_geo_unit, DF_product_matching, on=["ProductTypeName"], how="inner")
print(EX_geo_unit_product.shape)
EX_geo_unit_product.head(2)

(471019, 14)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography,unitName,name
0,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg,Wheat,AU,kg,wheat grain
1,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg,Wheat,AU,kg,"wheat grain, feed"


In [64]:
EX_geo_unit_process = pd.merge(EX_geo_unit, DF_process_matching, on=["ProductTypeName"], how="inner")
print(EX_geo_unit_process.shape)
EX_geo_unit_process.head(2)

(936452, 14)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography,unitName,key
0,96,AU,Production of electricity by coal,c40.11.a,P_POWC,TJ,361,MJ,0.361,kg CO2-eq/MJ,Electricity by coal,AU,MJ,"('ecoinvent 3.4 cutoff', '58353379c2ac55bb6bb0..."
1,96,AU,Production of electricity by coal,c40.11.a,P_POWC,TJ,361,MJ,0.361,kg CO2-eq/MJ,Electricity by coal,AU,MJ,"('ecoinvent 3.4 cutoff', '6f335a441b837359e503..."


In [65]:
EX_geo_unit_process["PhysicalUnit"].unique()

array(['MJ'], dtype=object)

In [66]:
# checks... 
#E = EX_geo_product_process_unit
#print(E.shape)
#E[(E["key"].notnull() & E["unitName"].notnull() )]

## Matching (the Pandas way)

In [67]:
M1 = pd.merge(EX_geo_unit_product, AO, on=["geography", "name", "unitName"], how="inner")
M1.apply(str)
numeric_columns = ["CarbonFootprint", "('IPCC 2013', 'climate change', 'GWP 100a')"]
M1[numeric_columns] = M1[numeric_columns].apply(pd.to_numeric)
print(M1.shape)
M1.head(2)

(2812, 68)


,#_x,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography,unitName,name,key,#_y,id,filename,activityName,startDate,endDate,specialActivityType,technologyLevel,inheritance status,parentActivityId,tags,synonyms,ISIC v4,ISIC code,ISIC name,specialty production,constrained market,accessRestrictedTo,dataEntryBy,dataGenerator,isActiveAuthor,group,CPC,amount,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"('IPCC 2013', 'climate change', 'GWP 100a')_all_CFs","('IPCC 2013', 'climate change', 'GWP 100a')",Unnamed: 0,#activities,Code (ecoinvent),Description (ecoinvent),ISIC level,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
0,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg,Wheat,AU,kg,wheat grain,"('ecoinvent 3.4 cutoff', 'd0bd9c43e8becfcc1a3d...",3759,40d2fd8e-c1c6-421a-a7e7-0723c4599f56,40d2fd8e-c1c6-421a-a7e7-0723c4599f56_0201ee86-...,wheat production,2009-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,3be71d91-dd7a-48f2-80a1-49cdea26814c,WFLDB,NaN,"0111:Growing of cereals (except rice), legumin...",111,"Growing of cereals (except rice), leguminous c...",False,NaN,1,Patrik Mouron,Eliane Riedener,True,ReferenceProduct,"01112: Wheat, other",1,2.53e+10,True,"Source: FAOSTAT. Average, 2009-2012",0.131,allocatable product,non-mft,1,1.64,0.656,0,205,0111,"Growing of cereals (except rice), leguminous c...",4,A,1.0,11.0,111.0,nan,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",Growing of non-perennial crops,"Growing of cereals (except rice), leguminous c...",nan,"A:Agriculture, forestry and fishing","01:Crop and animal production, hunting and rel...",011:Growing of non-perennial crops,"0111:Growing of cereals (except rice), legumin...",nan
1,822,CA,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.327,kg,0.327,kg CO2-eq/kg,Wheat,Canada without Quebec,kg,wheat grain,"('ecoinvent 3.4 cutoff', 'f053a42fddfcd4278d97...",14158,f34462a8-dcd0-4706-ac4b-aa748b36a0f4,f34462a8-dcd0-4706-ac4b-aa748b36a0f4_0201ee86-...,wheat production,2009-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,16b1f765-0dd1-4660-841c-efa7c6a5a8cd,WFLDB,NaN,"0111:Growing of cereals (except rice), legumin...",111,"Growing of cereals (except rice), leguminous c...",False,NaN,1,Patrik Mouron,Eliane Riedener,True,ReferenceProduct,"01112: Wheat, other",1,2.56e+10,True,"Source: FAOSTAT. Average, 2009-2012",0.131,allocatable product,non-mft,1,0.503,0.501,0,205,0111,"Growing of cereals (except rice), leguminous c...",4,A,1.0,11.0,111.0,nan,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",Growing of non-perennial crops,"Growing of cereals (except rice), leguminous c...",nan,"A:Agriculture, forestry and fishing","01:Crop and animal production, hunting and rel...",011:Growing of non-perennial crops,"0111:Growing of cereals (except rice), legumin...",nan


In [68]:
# check
M1[(M1['ProductTypeName'] == 'Cattle') & (M1['CountryCode'] == 'CA')][['GHGs-eq - tonnes', 'CarbonFootprint', "('IPCC 2013', 'climate change', 'GWP 100a')"]]

,GHGs-eq - tonnes,CarbonFootprint,"('IPCC 2013', 'climate change', 'GWP 100a')"
317,18,18,9.36


In [69]:
M2 = pd.merge(EX_geo_unit_process, AO, on=["geography", "key", "unitName"], how="inner")
M2.apply(str)
numeric_columns = ["CarbonFootprint", "('IPCC 2013', 'climate change', 'GWP 100a')"]
M2[numeric_columns] = M2[numeric_columns].apply(pd.to_numeric)
print(M2.shape)
M2.head(2)

(1755, 68)


,#_x,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography,unitName,key,#_y,id,filename,activityName,startDate,endDate,specialActivityType,technologyLevel,inheritance status,parentActivityId,tags,synonyms,ISIC v4,ISIC code,ISIC name,specialty production,constrained market,accessRestrictedTo,dataEntryBy,dataGenerator,isActiveAuthor,group,name,CPC,amount,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"('IPCC 2013', 'climate change', 'GWP 100a')_all_CFs","('IPCC 2013', 'climate change', 'GWP 100a')",Unnamed: 0,#activities,Code (ecoinvent),Description (ecoinvent),ISIC level,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
0,96,AU,Production of electricity by coal,c40.11.a,P_POWC,TJ,361,MJ,0.361,kg CO2-eq/MJ,Electricity by coal,AU,kWh,"('ecoinvent 3.4 cutoff', 'c4d3dcc31c97a01ef7b1...",14092,f2513c58-268c-4b43-a617-f227ca3ac5a1,f2513c58-268c-4b43-a617-f227ca3ac5a1_66c93e71-...,"electricity production, hard coal",1980-01-01,2017-12-31,ordinary transforming activity,Modern,allocated dataset,24731d86-38a5-40a5-a949-9b213556ac7f,hard coal power\n fossil fuels\n coal power,coking coal\n anthracite\n other bituminous coal,"3510:Electric power generation, transmission a...",3510,"Electric power generation, transmission and di...",True,NaN,1,Karin Treyer,Karin Treyer,True,ReferenceProduct,"electricity, high voltage",17100: Electrical energy,1,3.04e+10,True,Calculated value,0.0977,allocatable product,non-mft,1,1.11,1.11,138,2817,3510,"Electric power generation, transmission and di...",4,D,35.0,351.0,3510.0,nan,"Electricity, gas, steam and air conditioning s...","Electricity, gas, steam and air conditioning s...","Electric power generation, transmission and di...","Electric power generation, transmission and di...",nan,"D:Electricity, gas, steam and air conditioning...","35:Electricity, gas, steam and air conditionin...","351:Electric power generation, transmission an...","3510:Electric power generation, transmission a...",nan
1,96,AU,Production of electricity by coal,c40.11.a,P_POWC,TJ,361,MJ,0.361,kg CO2-eq/MJ,Electricity by coal,AU,kWh,"('ecoinvent 3.4 cutoff', '3a54ec6f33bd5af8782a...",11601,c6d76656-4cdb-4efd-82a2-95ee3a5f67ab,c6d76656-4cdb-4efd-82a2-95ee3a5f67ab_66c93e71-...,"electricity production, lignite",1980-01-01,2017-12-31,ordinary transforming activity,Modern,allocated dataset,9ec806d6-e465-4995-b573-980122e5b816,coal power\n lignite power\n fossil fuels\n,brown coal\n sub-bituminous coal,"3510:Electric power generation, transmission a...",3510,"Electric power generation, transmission and di...",True,NaN,1,Karin Treyer,Karin Treyer,True,ReferenceProduct,"electricity, high voltage",17100: Electrical energy,1,1.28e+11,True,Calculated value,0.0977,allocatable product,non-mft,1,1.24,1.24,138,2817,3510,"Electric power generation, transmission and di...",4,D,35.0,351.0,3510.0,nan,"Electricity, gas, steam and air conditioning s...","Electricity, gas, steam and air conditioning s...","Electric power generation, transmission and di...","Electric power generation, transmission and di...",nan,"D:Electricity, gas, steam and air conditioning...","35:Electricity, gas, steam and air conditionin...","351:Electric power generation, transmission an...","3510:Electric power generation, transmission a...",nan


In [70]:
M = pd.concat([M1, M2], sort=False)
M = M.reset_index()
print(M.shape)
M.head(2)

(4567, 69)


,index,#_x,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography,unitName,name,key,#_y,id,filename,activityName,startDate,endDate,specialActivityType,technologyLevel,inheritance status,parentActivityId,tags,synonyms,ISIC v4,ISIC code,ISIC name,specialty production,constrained market,accessRestrictedTo,dataEntryBy,dataGenerator,isActiveAuthor,group,CPC,amount,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"('IPCC 2013', 'climate change', 'GWP 100a')_all_CFs","('IPCC 2013', 'climate change', 'GWP 100a')",Unnamed: 0,#activities,Code (ecoinvent),Description (ecoinvent),ISIC level,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
0,0,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg,Wheat,AU,kg,wheat grain,"('ecoinvent 3.4 cutoff', 'd0bd9c43e8becfcc1a3d...",3759,40d2fd8e-c1c6-421a-a7e7-0723c4599f56,40d2fd8e-c1c6-421a-a7e7-0723c4599f56_0201ee86-...,wheat production,2009-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,3be71d91-dd7a-48f2-80a1-49cdea26814c,WFLDB,NaN,"0111:Growing of cereals (except rice), legumin...",111,"Growing of cereals (except rice), leguminous c...",False,NaN,1,Patrik Mouron,Eliane Riedener,True,ReferenceProduct,"01112: Wheat, other",1,2.53e+10,True,"Source: FAOSTAT. Average, 2009-2012",0.131,allocatable product,non-mft,1,1.64,0.656,0,205,0111,"Growing of cereals (except rice), leguminous c...",4,A,1.0,11.0,111.0,nan,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",Growing of non-perennial crops,"Growing of cereals (except rice), leguminous c...",nan,"A:Agriculture, forestry and fishing","01:Crop and animal production, hunting and rel...",011:Growing of non-perennial crops,"0111:Growing of cereals (except rice), legumin...",nan
1,1,822,CA,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.327,kg,0.327,kg CO2-eq/kg,Wheat,Canada without Quebec,kg,wheat grain,"('ecoinvent 3.4 cutoff', 'f053a42fddfcd4278d97...",14158,f34462a8-dcd0-4706-ac4b-aa748b36a0f4,f34462a8-dcd0-4706-ac4b-aa748b36a0f4_0201ee86-...,wheat production,2009-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,16b1f765-0dd1-4660-841c-efa7c6a5a8cd,WFLDB,NaN,"0111:Growing of cereals (except rice), legumin...",111,"Growing of cereals (except rice), leguminous c...",False,NaN,1,Patrik Mouron,Eliane Riedener,True,ReferenceProduct,"01112: Wheat, other",1,2.56e+10,True,"Source: FAOSTAT. Average, 2009-2012",0.131,allocatable product,non-mft,1,0.503,0.501,0,205,0111,"Growing of cereals (except rice), leguminous c...",4,A,1.0,11.0,111.0,nan,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",Growing of non-perennial crops,"Growing of cereals (except rice), leguminous c...",nan,"A:Agriculture, forestry and fishing","01:Crop and animal production, hunting and rel...",011:Growing of non-perennial crops,"0111:Growing of cereals (except rice), legumin...",nan


In [71]:
M.index

RangeIndex(start=0, stop=4567, step=1)

In [72]:
# check
check_columns = [['GHGs-eq - tonnes', 'CarbonFootprint', "('IPCC 2013', 'climate change', 'GWP 100a')"]]
M[(M['ProductTypeName'] == 'Cattle') & (M['CountryCode'] == 'CA')]
M['CarbonFootprint'].sum()

155036.32570588624

In [73]:
# convert MJ to kWh 
index_kWh = M.loc[M.unitName == "kWh"].index
#print(M.loc[317])
print("Converted", len(index_kWh), "values from MJ to kWh.")
M["CarbonFootprint"].loc[index_kWh] = M["CarbonFootprint"].loc[index_kWh] * 3.6
M.head(2)

Converted 1755 values from MJ to kWh.


C:\Users\steub\miniconda3\envs\ab38\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,index,#_x,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography,unitName,name,key,#_y,id,filename,activityName,startDate,endDate,specialActivityType,technologyLevel,inheritance status,parentActivityId,tags,synonyms,ISIC v4,ISIC code,ISIC name,specialty production,constrained market,accessRestrictedTo,dataEntryBy,dataGenerator,isActiveAuthor,group,CPC,amount,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"('IPCC 2013', 'climate change', 'GWP 100a')_all_CFs","('IPCC 2013', 'climate change', 'GWP 100a')",Unnamed: 0,#activities,Code (ecoinvent),Description (ecoinvent),ISIC level,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
0,0,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg,Wheat,AU,kg,wheat grain,"('ecoinvent 3.4 cutoff', 'd0bd9c43e8becfcc1a3d...",3759,40d2fd8e-c1c6-421a-a7e7-0723c4599f56,40d2fd8e-c1c6-421a-a7e7-0723c4599f56_0201ee86-...,wheat production,2009-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,3be71d91-dd7a-48f2-80a1-49cdea26814c,WFLDB,NaN,"0111:Growing of cereals (except rice), legumin...",111,"Growing of cereals (except rice), leguminous c...",False,NaN,1,Patrik Mouron,Eliane Riedener,True,ReferenceProduct,"01112: Wheat, other",1,2.53e+10,True,"Source: FAOSTAT. Average, 2009-2012",0.131,allocatable product,non-mft,1,1.64,0.656,0,205,0111,"Growing of cereals (except rice), leguminous c...",4,A,1.0,11.0,111.0,nan,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",Growing of non-perennial crops,"Growing of cereals (except rice), leguminous c...",nan,"A:Agriculture, forestry and fishing","01:Crop and animal production, hunting and rel...",011:Growing of non-perennial crops,"0111:Growing of cereals (except rice), legumin...",nan
1,1,822,CA,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.327,kg,0.327,kg CO2-eq/kg,Wheat,Canada without Quebec,kg,wheat grain,"('ecoinvent 3.4 cutoff', 'f053a42fddfcd4278d97...",14158,f34462a8-dcd0-4706-ac4b-aa748b36a0f4,f34462a8-dcd0-4706-ac4b-aa748b36a0f4_0201ee86-...,wheat production,2009-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,16b1f765-0dd1-4660-841c-efa7c6a5a8cd,WFLDB,NaN,"0111:Growing of cereals (except rice), legumin...",111,"Growing of cereals (except rice), leguminous c...",False,NaN,1,Patrik Mouron,Eliane Riedener,True,ReferenceProduct,"01112: Wheat, other",1,2.56e+10,True,"Source: FAOSTAT. Average, 2009-2012",0.131,allocatable product,non-mft,1,0.503,0.501,0,205,0111,"Growing of cereals (except rice), leguminous c...",4,A,1.0,11.0,111.0,nan,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",Growing of non-perennial crops,"Growing of cereals (except rice), leguminous c...",nan,"A:Agriculture, forestry and fishing","01:Crop and animal production, hunting and rel...",011:Growing of non-perennial crops,"0111:Growing of cereals (except rice), legumin...",nan


In [74]:
# check
M[(M['ProductTypeName'] == 'Cattle') & (M['CountryCode'] == 'CA')][['GHGs-eq - tonnes', 'CarbonFootprint', "('IPCC 2013', 'climate change', 'GWP 100a')"]]
M['CarbonFootprint'].sum()

155482.2792929991

In [75]:
# drop rows where EX CarbonFootprint == 0
ind = M.loc[M["CarbonFootprint"] == 0].index
print("Excluding", len(ind), "'zero' values.")
M.drop(ind, inplace=True)
print(M.shape)

Excluding 0 'zero' values.
(4567, 69)


In [76]:
# check
M[(M['ProductTypeName'] == 'Cattle') & (M['CountryCode'] == 'CA')][['GHGs-eq - tonnes', 'CarbonFootprint', "('IPCC 2013', 'climate change', 'GWP 100a')"]]

,GHGs-eq - tonnes,CarbonFootprint,"('IPCC 2013', 'climate change', 'GWP 100a')"
317,18,18,9.36


## Add continents

In [77]:
#DF_geo = pd.read_excel(f"{DATA_PATH}matching/matching_ecoinvent_Exiobase_geographies.xlsx", sheet_name='matched_191117', skiprows=1)
#DF_geo.rename(columns={'Exiobase 3': 'CountryCode'}, inplace=True)
#DF_geo = DF_geo[['Continent', 'CountryCode']].drop_duplicates(keep=False)
#DF_geo.set_index('CountryCode', inplace=True)
#print(DF_geo.shape)
#DF_geo.head(2)

In [78]:
# we use ecoinvent geographies
#DF_geo = pd.read_excel(f"{DATA_PATH}matching/matching_ecoinvent_Exiobase_geographies.xlsx", sheet_name='matched_191117', skiprows=1)
DF_geo = GEO
DF_geo.rename(columns={'ecoinvent 3.4': 'geography'}, inplace=True)
DF_geo = DF_geo[['Continent', 'geography']]  #.drop_duplicates(keep=False)
DF_geo.set_index('geography', inplace=True)
print(DF_geo.shape)
DF_geo.head(2)

(261, 1)


,Continent
geography,
AE,Asia
AL,Europe


In [79]:
M = M.join(DF_geo, on=['geography'], how='left')
M.shape

(4567, 70)

### add ISIC 1 information where it is missing

In [80]:
type(M[M['ProductTypeName'] == 'Manure (conventional treatment)']['ISIC 1'].values[0])

str

In [81]:
M[M['ISIC 1'] == "nan"].head()

,index,#_x,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography,unitName,name,key,#_y,id,filename,activityName,startDate,endDate,specialActivityType,technologyLevel,inheritance status,parentActivityId,tags,synonyms,ISIC v4,ISIC code,ISIC name,specialty production,constrained market,accessRestrictedTo,dataEntryBy,dataGenerator,isActiveAuthor,group,CPC,amount,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"('IPCC 2013', 'climate change', 'GWP 100a')_all_CFs","('IPCC 2013', 'climate change', 'GWP 100a')",Unnamed: 0,#activities,Code (ecoinvent),Description (ecoinvent),ISIC level,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5,Continent
250,250,7879,GLO,Cultivation of plant-based fibers,c01.g,P_FIBR,tonnes,1.59,kg,1.59,kg CO2-eq/kg,Plant-based fibers,GLO,kg,coconut husk,"('ecoinvent 3.4 cutoff', 'bb51d5d91410516e9aaf...",14830,ff02f2dc-6b11-4037-8e1e-542147379d21,ff02f2dc-6b11-4037-8e1e-542147379d21_835bdfad-...,"coconut husk, Recycled Content cut-off",2017-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,NaN,NaN,NaN,not available now:not available now,not available now,not available now,False,NaN,1,[System],[System],True,ReferenceProduct,"01913: Cereal straw, husks, unprepared, ground...",1,0,False,NaN,0,Recyclable,non-mft,1,0,0,185,163,nan,not available now,17,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Global
327,327,7888,GLO,"Manure treatment (conventional), storage and l...",c01.w.1,P_MANC,tonnes,0.242,kg,0.242,kg CO2-eq/kg,Manure (conventional treatment),GLO,kg,"manure, liquid, cattle","('ecoinvent 3.4 cutoff', 'd5d3559ca53fb23c22bb...",9940,aa9becc0-421b-49f1-89ee-6a3b1a6777de,aa9becc0-421b-49f1-89ee-6a3b1a6777de_ad551fe0-...,"manure, liquid, cattle, Recycled Content cut-off",2017-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,NaN,NaN,NaN,not available now:not available now,not available now,not available now,False,NaN,1,[System],[System],True,ReferenceProduct,34654: Excreta of animals useful for manure/fe...,1,0,False,NaN,0,Recyclable,non-mft,1,0,0,185,163,nan,not available now,17,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Global
328,328,7888,GLO,"Manure treatment (conventional), storage and l...",c01.w.1,P_MANC,tonnes,0.242,kg,0.242,kg CO2-eq/kg,Manure (conventional treatment),GLO,kg,"manure, liquid, swine","('ecoinvent 3.4 cutoff', '840b95f5c81c831cd94c...",10918,bac8f06d-d7be-4331-b541-8bacd978985a,bac8f06d-d7be-4331-b541-8bacd978985a_6c50f204-...,"manure, liquid, swine, Recycled Content cut-off",2017-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,NaN,NaN,NaN,not available now:not available now,not available now,not available now,False,NaN,1,[System],[System],True,ReferenceProduct,34654: Excreta of animals useful for manure/fe...,1,0,False,NaN,0,Recyclable,non-mft,1,0,0,185,163,nan,not available now,17,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Global
329,329,7888,GLO,"Manure treatment (conventional), storage and l...",c01.w.1,P_MANC,tonnes,0.242,kg,0.242,kg CO2-eq/kg,Manure (conventional treatment),GLO,kg,"manure, solid, cattle","('ecoinvent 3.4 cutoff', '24e07826db670824d951...",10815,b8dc5812-becf-4c34-bd73-798c498b7ce5,b8dc5812-becf-4c34-bd73-798c498b7ce5_ed56bc72-...,"manure, solid, cattle, Recycled Content cut-off",2017-01-01,2017-12-31,ordinary transforming activity,Current,allocated dataset,NaN,NaN,NaN,not available now:not available now,not available now,not available now,False,NaN,1,[System],[System],True,ReferenceProduct,34654: Excreta of animals useful for manure/fe...,1,0,False,NaN,0,Recyclable,non-mft,1,0,0,185,163,nan,not available now,17,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Global
639,639,7925,

In [88]:
M[M['ProductTypeName'] == '']

,index,#_x,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,ProductTypeName,geography,unitName,name,key,#_y,id,filename,activityName,startDate,endDate,specialActivityType,technologyLevel,inheritance status,parentActivityId,tags,synonyms,ISIC v4,ISIC code,ISIC name,specialty production,constrained market,accessRestrictedTo,dataEntryBy,dataGenerator,isActiveAuthor,group,CPC,amount,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"('IPCC 2013', 'climate change', 'GWP 100a')_all_CFs","('IPCC 2013', 'climate change', 'GWP 100a')",Unnamed: 0,#activities,Code (ecoinvent),Description (ecoinvent),ISIC level,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5,Continent


## Add relative deviation and difference

In [89]:
M["diff"] = M["CarbonFootprint"] - M["('IPCC 2013', 'climate change', 'GWP 100a')"]
# difference of ecoinvent result from EXIOBASE (1 = perfect match; 0 = no ecoinvent score; 2 = twice the impact)
#M["diff%"] = 1 - (M["CarbonFootprint"] - M["('IPCC 2013', 'climate change', 'GWP 100a')"])/M["CarbonFootprint"]
M["dev"] = M["('IPCC 2013', 'climate change', 'GWP 100a')"]/M["CarbonFootprint"] - 1.0

# Save matching results

In [90]:
M.columns

Index(['index', '#_x', 'CountryCode', 'ProductTypeName_of_hiot',
       'product code 1', 'product code 2', 'OriginalPhysicalUnit',
       'GHGs-eq - tonnes', 'PhysicalUnit', 'CarbonFootprint', 'unit',
       'ProductTypeName', 'geography', 'unitName', 'name', 'key', '#_y', 'id',
       'filename', 'activityName', 'startDate', 'endDate',
       'specialActivityType', 'technologyLevel', 'inheritance status',
       'parentActivityId', 'tags', 'synonyms', 'ISIC v4', 'ISIC code',
       'ISIC name', 'specialty production', 'constrained market',
       'accessRestrictedTo', 'dataEntryBy', 'dataGenerator', 'isActiveAuthor',
       'group', 'CPC', 'amount', 'productionVolumeAmount', 'absolute PV',
       'productionVolumeComment', 'price', 'By-product classification', 'mft',
       'functional unit',
       '('IPCC 2013', 'climate change', 'GWP 100a')_all_CFs',
       '('IPCC 2013', 'climate change', 'GWP 100a')', 'Unnamed: 0',
       '#activities', 'Code (ecoinvent)', 'Description (ecoinven

In [91]:
reduced_index = ["key",
"functional unit",
"unitName",
"PhysicalUnit",
"ProductTypeName_of_hiot",
"unit",
"ISIC 1",
"ISIC 2",
"ISIC 3",
"ISIC 4",
"ISIC v4",
"CPC",
"activityName",
"name",
"ProductTypeName",
"geography",
"CountryCode",
"Continent",
"('IPCC 2013', 'climate change', 'GWP 100a')",
"CarbonFootprint",
"diff",
"dev",]

In [92]:
# save EXCEL
hf.append_df_to_excel(M[reduced_index], 
                      template=f"{DATA_PATH}matching_results/template_matching.xlsx",
                      filename=f"{SCENARIO_PATH}matching_results/EItoEX_matching_results_HIOT_manual.xlsx", 
                      sheet_name='data',
                      truncate_sheet=False,
                      startrow=1)